In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit
import astropy
import lightkurve as lk
from scipy.optimize import minimize
from lightkurve import search_lightcurvefile
from lightkurve import search_lightcurve

from astropy.table import Table, join, MaskedColumn, vstack, Column

import sys  
sys.path.append('/Users/Tobin/Dropbox/Stellar_Flares_Project/Lupita_Flare_Model/Llamaradas-Estelares/') #Edit this to your own file path
from Flare_model import flare_model

import astropy.units as u

sys.path.append('/Users/Tobin/Dropbox/Stellar_Flares_Project/Getting_Started/')
import stella
from tqdm import tqdm_notebook
import os, sys
from stella.download_nn_set import DownloadSets

/Users/Tobin/opt/anaconda3/envs/Jim/lib/python3.11/site-packages/lightkurve/config/__init__.py:119: UserWarning: The default Lightkurve cache directory, used by download(), etc., has been moved to /Users/Tobin/.lightkurve/cache. Please move all the files in the legacy directory /Users/Tobin/.lightkurve-cache to the new location and remove the legacy directory. Refer to https://docs.lightkurve.org/reference/config.html#default-cache-directory-migration for more information.
  warnings.warn(
2023-12-12 15:01:16.877714: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [16]:
lupita_tab=Table.read('lupita_tab_3.mrt', format='mrt')
lupita_tab

labeling_flare=np.zeros(len(lupita_tab))

for i in range(len(lupita_tab)):
    labeling_flare[i]=int(i)
    
lupita_tab.add_column(Column(labeling_flare.astype(int)), name='ID', index=0)

lupita_tab

ID,tpeak,FWHM,Amp,e_tpeak,e_FWHM,e_Amp,Source
,d,d,,d,d,,
int64,float64,float64,float64,str11,str11,float64,str6
0,539.6503088,0.00153389,0.008934698,6.49E-05,0.000213685,0.000716695,Kepler
1,539.8699037,0.000956397,0.01223805,8.61E-05,0.000185378,0.00213297,Kepler
2,541.1449682,0.002426536,0.007813247,5.18E-05,0.000187954,0.00037484,Kepler
3,541.1731539,0.003951658,0.011726901,0.000102912,0.000339299,0.000683433,Kepler
...,...,...,...,...,...,...,...
435,1725.532746,0.006001372,0.015728469,0.000293083,0.001023428,0.00174061,TESS
436,1726.595741,0.004285892,0.009459497,0.000249091,0.000854394,0.001214612,TESS
437,1726.84291,0.001154102,0.047080266,0.000278262,0.00062281,0.028681623,TESS


In [21]:
import pickle

flare_info=pickle.load(open("/Users/Tobin/Dropbox/Stellar_Flares_Project/Getting_Started/Energies_and_rates.pkl", 'rb'))


In [61]:
def get_lc(tic, sector_ind):

    search = lk.search_lightcurve(target=tic, mission='TESS', author='SPOC')

    search.table["dataURL"] = search.table["dataURI"]
    lc = search[sector_ind].download()
    lc=lc[~np.isnan(lc.flux.value)]
    
    
    return lc

def syn_flare_insertion(lc, flare_amp, flare_fwhm, inserted_time_step):
    
    model_flare_flux=flare_model(lc['time'].value, lc['time'].value[inserted_time_step], 
                             flare_fwhm, flare_amp*np.median(lc.flux.value))
    
    
    
    fixed_mask=np.ma.filled(model_flare_flux, fill_value=0)
    
    fixed_mask2=np.nan_to_num(fixed_mask, nan=0)
    
    lc_with_inserted_flare=fixed_mask2+lc.flux.value
    
    return lc_with_inserted_flare


def recovery(lc, inserted_flare_flux, inserted_time_step):
    "Initialize Stella"
    OUT_DIR='/Users/Tobin/Dropbox/Stellar_Flares_Project/Getting Started/Results/'

    cnn = stella.ConvNN(output_dir=OUT_DIR)

    ds = DownloadSets()
    ds.download_models()

    MODELS=ds.models
    
    #Test recovery:
    preds = np.zeros((len(MODELS),len(lc['time'].value)))

    for i, model in enumerate(MODELS):
        cnn.predict(modelname=model,
                times=lc.time.value,
                fluxes=inserted_flare_flux,
                errs=lc['flux_err'].value)
        preds[i] = cnn.predictions[0]

    avg_pred = np.nanmedian(preds, axis=0)
    
    pred_at_inserterd_timestep = avg_pred[inserted_time_step]
    
    one_before=avg_pred[inserted_time_step-1]
    one_after=avg_pred[inserted_time_step+1]
    
    if pred_at_inserterd_timestep > 0.3 and one_before > 0.3 and one_after > 0.3:
        return True, avg_pred
    else:
        return False, None

In [64]:
def Injecting_and_recovery(tic, flare, sector_ind):
    
    lc = get_lc(tic, sector_ind)
    
    sector_bool=flare_info['Flare_Bool'][sector_ind]
    
    while True:
        rand_insertion_point=np.random.randint(2, len(lc)-3, size=1)
        
        print(rand_insertion_point, ~flare_info['Flare_Bool'][0][rand_insertion_point])
        
        if ~flare_info['Flare_Bool'][0][rand_insertion_point]:
            break

    inserted_flare=syn_flare_insertion(lc, lupita_tab['Amp'][flare], lupita_tab['FWHM'][flare], rand_insertion_point)

    recovered, pred = recovery(lc, inserted_flare, rand_insertion_point)

    return recovered

In [36]:
list_of_rows=[]

for i in range(len(lupita_tab)):
    row = Table(lupita_tab[i])
    
    row.add_column(Column([0]), name='Sector_Insertion_Trial', index=1)
    
    row.add_column(Column([0]), name='Sector', index=1)
    
    list_of_10_rows=[]
    
    for j in range(10):
        list_of_10_rows.append(row)
        
    table_of_length_10=vstack(list_of_10_rows)
    
    for j in range(len(table_of_length_10)):
        table_of_length_10['Sector_Insertion_Trial'][j]=j+1
    
    list_of_50_rows=[]
    
    for k in range(5):
        list_of_50_rows.append(table_of_length_10)
    
    table_of_length_50=vstack(list_of_50_rows)
    
    for j in range(len(table_of_length_50)):
        if j <10:
            table_of_length_50['Sector'][j]=0
        if j >10 and j < 20:
            table_of_length_50['Sector'][j]=1
        if j >20 and j < 30:
            table_of_length_50['Sector'][j]=2
        if j >30 and j < 40:
            table_of_length_50['Sector'][j]=3
        if j > 40:
            table_of_length_50['Sector'][j]=4
            
    
    list_of_rows.append(table_of_length_50)
        
trial_table=vstack(list_of_rows)
trial_table

Recovery=np.zeros(len(trial_table))

trial_table.add_column(Column(Recovery), name='Recovered', index=3)

trial_table['Recovered'] = trial_table['Recovered'].astype('bool')

trial_table

ID,Sector,Sector_Insertion_Trial,Recovered,tpeak,FWHM,Amp,e_tpeak,e_FWHM,e_Amp,Source
,,,,d,d,,d,d,,
int64,int64,int64,bool,float64,float64,float64,str11,str11,float64,str6
0,0,1,False,539.6503088,0.00153389,0.008934698,6.49E-05,0.000213685,0.000716695,Kepler
0,0,2,False,539.6503088,0.00153389,0.008934698,6.49E-05,0.000213685,0.000716695,Kepler
0,0,3,False,539.6503088,0.00153389,0.008934698,6.49E-05,0.000213685,0.000716695,Kepler
0,0,4,False,539.6503088,0.00153389,0.008934698,6.49E-05,0.000213685,0.000716695,Kepler
...,...,...,...,...,...,...,...,...,...,...
438,4,7,False,1720.841125,0.00298599,0.005046997,0.000754882,0.001403928,0.001474787,TESS
438,4,8,False,1720.841125,0.00298599,0.005046997,0.000754882,0.001403928,0.001474787,TESS
438,4,9,False,1720.841125,0.00298599,0.005046997,0.000754882,0.001403928,0.001474787,TESS


In [65]:
#testing

for i in range(5):
    trial_table[i]['Recovered']=Injecting_and_recovery('tic272272592', trial_table[i]['ID'], trial_table[i]['Sector'])
    
trial_table[:5]

[13304] [ True]
Can only use stella.ConvNN.predict().
Models have already been downloaded to ~/.stella/models


/Users/Tobin/Dropbox/Stellar_Flares_Project/Lupita_Flare_Model/Llamaradas-Estelares/Flare_model.py:21: RuntimeWarning: overflow encountered in exp
  eqn = ((1 / 2) * np.sqrt(np.pi) * A * C * f1 * np.exp(-D1 * t + ((B / C) + (D1 * C / 2)) ** 2)
/Users/Tobin/Dropbox/Stellar_Flares_Project/Lupita_Flare_Model/Llamaradas-Estelares/Flare_model.py:21: RuntimeWarning: invalid value encountered in multiply
  eqn = ((1 / 2) * np.sqrt(np.pi) * A * C * f1 * np.exp(-D1 * t + ((B / C) + (D1 * C / 2)) ** 2)
/Users/Tobin/Dropbox/Stellar_Flares_Project/Lupita_Flare_Model/Llamaradas-Estelares/Flare_model.py:23: RuntimeWarning: overflow encountered in exp
  * np.exp(-D2 * t+ ((B / C) + (D2 * C / 2)) ** 2) * special.erfc(((B - t) / C) + (C * D2 / 2)))
/Users/Tobin/Dropbox/Stellar_Flares_Project/Lupita_Flare_Model/Llamaradas-Estelares/Flare_model.py:22: RuntimeWarning: invalid value encountered in multiply
  * special.erfc(((B - t) / C) + (C * D1 / 2))) + ((1 / 2) * np.sqrt(np.pi) * A * C * f2
  0%|       

546/546 [==============================] - 2s 3ms/step


  0%|                        | 0/1 [00:00<?, ?it/s]

546/546 [==============================] - 2s 2ms/step


...

546/546 [==============================] - 1s 3ms/step


  0%|                        | 0/1 [00:00<?, ?it/s]

546/546 [==============================] - 2s 3ms/step


100%|████████████████| 1/1 [00:01<00:00,  1.96s/it]


ID,Sector,Sector_Insertion_Trial,Recovered,tpeak,FWHM,Amp,e_tpeak,e_FWHM,e_Amp,Source
,,,,d,d,,d,d,,
int64,int64,int64,bool,float64,float64,float64,str11,str11,float64,str6
0,0,1,False,539.6503088,0.00153389,0.008934698,6.49E-05,0.000213685,0.000716695,Kepler
0,0,2,False,539.6503088,0.00153389,0.008934698,6.49E-05,0.000213685,0.000716695,Kepler
0,0,3,False,539.6503088,0.00153389,0.008934698,6.49E-05,0.000213685,0.000716695,Kepler
0,0,4,False,539.6503088,0.00153389,0.008934698,6.49E-05,0.000213685,0.000716695,Kepler
0,0,5,False,539.6503088,0.00153389,0.008934698,6.49E-05,0.000213685,0.000716695,Kepler


In [ ]:
#Read in file from .py script

trial_table = Table.read('Injection_Recovery_Table_Results.fits')

trial_table

In [ ]:
#Calc the EDs for each of the flares 